In [1]:
import json
import regex as re 
import requests 
import sys
import pathlib
import os
import time

utils_dir = pathlib.Path.cwd().parent
sys.path.append(str(utils_dir)+"\\")
import utils
import natsort
import threading
import math

import numpy as np

In [2]:
with open(utils_dir / r"topics.json","r") as in_json:
    topics = json.load(in_json)



#alias for claude sonnet 4 
model = "claude-sonnet-4-0"
json_name = "metadata.json"
interaction_info = {}
context_window = 190000
max_completion_length = 64000
novel_desired_length = 150000
num_novels = 1000
chapter_length = 15000
#per minute
RPM_limit = 2000
ITPM_limit = 800000
OTPM_limit = 160000
requests_sent = 0
itokens = 0
otokens = 0
error_js_name = "error.json"
text_out_dir = r"generated_text"
interaction_info = {}

thread_pause_semaphore = threading.Semaphore(1)
if(json_name in os.listdir(os.getcwd())):
        with open(json_name,"r") as infile:
            interaction_info = json.load(infile)
    
def request_anthropic(prompt, model, api_key, max_tokens=chapter_length):
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "x-api-key": f"{api_key}",
        "content-type": "application/json",
        "anthropic-version": "2023-06-01"
    }
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    print("request posted")
    response = requests.post(url, headers=headers, json=data)
    response_js = response.json()
    itokens = response_js["usage"]["input_tokens"]
    otokens = response_js["usage"]["output_tokens"]
    text =  response_js["content"][0]["text"]

    if response.status_code == 200:
        print(f"request returned")
        return response_js,text,itokens,otokens
    else:
        print(f"Error: {response.status_code}, {response.text}")
        sys.exit()


def check_for_error(response, error_js_name):
    if(response["stop_reason"]!= "end_turn"):
        print(f"invalid stop reason: {response["stop_reason"]}")
        json.dump(response, error_js_name)
        sys.exit()


In [5]:
metadata_out_dir = "metadata"
interaction_info = {}
if(not text_out_dir in os.listdir()):
    os.mkdir(text_out_dir)
    
files = utils.get_all_files_with_extension(f"{os.getcwd()}\\{metadata_out_dir}",".json")
files = natsort.natsorted(files)
print(f"files: {files}")
for file in files:
    with open(f"{os.getcwd()}\\{metadata_out_dir}\\{file}","r") as infile:
        interaction_info = json.load(infile)
    text = interaction_info["novel_text"]
    
    index = int(re.search("(\d+)",os.path.basename(file)).group(0))
    with open(f"{os.getcwd()}\\{text_out_dir}\\claude_sonnet{index}.txt","w",encoding="utf-8") as outfile:
        outfile.write(text)

<>:14: SyntaxWarning: invalid escape sequence '\d'
<>:14: SyntaxWarning: invalid escape sequence '\d'
C:\Users\7ross\AppData\Local\Temp\ipykernel_25832\981116066.py:14: SyntaxWarning: invalid escape sequence '\d'
  index = int(re.search("(\d+)",os.path.basename(file)).group(0))


files: ['metadata0.json', 'metadata1.json', 'metadata2.json', 'metadata3.json', 'metadata4.json', 'metadata5.json', 'metadata6.json', 'metadata7.json', 'metadata8.json', 'metadata9.json', 'metadata10.json', 'metadata11.json', 'metadata12.json', 'metadata13.json', 'metadata14.json', 'metadata15.json', 'metadata16.json', 'metadata17.json', 'metadata18.json', 'metadata19.json', 'metadata20.json', 'metadata21.json', 'metadata22.json', 'metadata23.json', 'metadata24.json', 'metadata25.json', 'metadata26.json', 'metadata27.json', 'metadata28.json', 'metadata29.json', 'metadata30.json', 'metadata31.json', 'metadata32.json', 'metadata33.json', 'metadata34.json', 'metadata35.json', 'metadata36.json', 'metadata37.json', 'metadata38.json', 'metadata39.json', 'metadata40.json', 'metadata41.json', 'metadata42.json', 'metadata43.json', 'metadata44.json', 'metadata45.json', 'metadata46.json', 'metadata47.json', 'metadata48.json', 'metadata49.json', 'metadata50.json', 'metadata51.json', 'metadata52.js

In [15]:
skip_indices = [8,68,69,76,85,119,126,132,133,174,181]
itokens = 0
otokens = 0
requests_sent = 0

itokens_semaphore = threading.Semaphore(1)
otokens_semaphore = threading.Semaphore(1)
requests_sent_semaphore = threading.Semaphore(1)


def check_rate_limits():
        #will block all other threads from generating until the calling thread is done waiting out the rate limiting
        global itokens
        global otokens
        global requests_sent
        with thread_pause_semaphore:
            if(itokens>ITPM_limit or otokens>OTPM_limit or requests_sent>RPM_limit):
                time.sleep(70)
                itokens = 0
                otokens = 0
                requests_sent = 0
@utils.multithreading
def generate_novel(topic:str,index:int) -> None:
    interaction_info = {}
    metadata_path = f"{os.getcwd()}\\{metadata}\\metadata{index}.json"
    global itokens 
    global otokens 
    global requests_sent
    
    
    if(index in skip_indices):
        print(f"novel index is in skipped indices: {index}")
        return

    if(not os.path.exists(metadata_path)):
        interaction_info["novel_text"] = ""
        interaction_info["responses"] = []
    else:
        print(f"found metadata at path {metadata_path}")
        with open(metadata_path,"r") as infile:
            interaction_info = json.load(infile)    
    

    novel_current_length = len(interaction_info["novel_text"])
    first_prompt = f"I want you to act as a novelist writing about {topic}. The total length of the novel is about {novel_desired_length} characters and after generating each section of the novel i will tell you how much of the text you have generated so far. Try to generate text with some irregularities in order to fool ai detection software. In the novel you should include plot points characters dialogue and an overall plot. Generate only the text for this novel and do not add any annotations. Pace the plot and story according to the current number of characters I tell you. Generate the text in full paragraphs and chapters of about {chapter_length} character length. Continue generating chapter 1 until I tell you to stop. Generate EXCLUSIVELY novel text with no annotations confirmations or listing the chapter number."
    
    
    while(len(interaction_info["novel_text"])<novel_desired_length):
        prompt = ""
        if(novel_current_length == 0):
            prompt = first_prompt
        else:
            prompt = interaction_info["novel_text"] + f"Here is the previous generated text. You have generated {novel_current_length} characters so far. Generate chapter {math.floor(novel_current_length/chapter_length)+1} until i tell you to stop, also remember to generate text in complete paragraphs and do not generate lines with a single sentence. Generate EXCLUSIVELY novel text"
            
            if(len(prompt)+max_completion_length>context_window):
                prompt = prompt[len(prompt)-(context_window-max_completion_length)::]
        check_rate_limits() 
        try: 
            response,text,itk,otk = request_anthropic(prompt,model,api_key,max_tokens=max_completion_length)
        except Exception as err:
            return err
        with requests_sent_semaphore:
            requests_sent += 1 
            
        check_for_error(response,error_js_name)
        with itokens_semaphore:
            itokens += itk
        with otokens_semaphore:
            otokens += otk
        interaction_info["responses"].append(response)
        interaction_info["novel_text"] += text
        novel_current_length += len(text)
        print(f"current novel: {index}")
        print(f"response text length: {len(text)}")
        print(f"total novel length: {novel_current_length}")
        print(f"preview: \n{text[:500]}")
        with open(metadata_path,"w") as outfile:
           json.dump(interaction_info,outfile,indent = 5) 

        if(otk < 500):
            print(f"adding {index} to skipped indices")
            skip_indices.append(index)
            return None
    return None



        

In [ ]:
def is_written(index:int) -> bool:
    metadata_path = f"{os.getcwd()}\\{text_out_dir}\\metadata{index}.json"
    if(os.path.exists(metadata_path)):
        interaction_info = {}
        with open(metadata_path,"r") as infile:
            interaction_info = json.load(infile)
        
        if(len(interaction_info["novel_text"])>=novel_desired_length):
            return True
        else:
            if(index in skip_indices):
                return True
            else:
                return False
    else:
        return False

def is_generation_finished():
    files = utils.get_all_files_with_extension(f"{os.getcwd()}\\{text_out_dir}",".json")
    files = natsort.natsorted(files)
    for file in files:
        index = int(re.search("(\d+)",os.path.basename(file)).group(0))
        if(not is_written(index)):
            return False


# writing = []
# for _ in range(num_novels):
#     novel_index = get_next_index()
#     if(novel_index == None):
#         break
#     else:
#         print(f"writing: {novel_index}")
#         writing.append(novel_index)
#         topic = topics[novel_index]
#         generate_novel(topic)
if(os.path.exists(f"{os.getcwd()}\\skip_indices.json")):
    with open(f"{os.getcwd()}\\skip_indices.json","r") as infile:
        skip_indices = json.load(infile)
for novel_index in range(num_novels):
    if(not is_written(novel_index)):
        topic = topics[novel_index]
        err = generate_novel(topic,novel_index)
        while(err != None):
            time.sleep(10)
            print(err)
            print("restarting generation")
            err = generate_novel(topic,novel_index)
        with open(f"{os.getcwd()}\\skip_indices.json","w") as outfile:
            json.dump(skip_indices,outfile)
    else:
        print(f"skipping index: {novel_index}")

<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
C:\Users\7ross\AppData\Local\Temp\ipykernel_20980\640172611.py:22: SyntaxWarning: invalid escape sequence '\d'
  index = int(re.search("(\d+)",os.path.basename(file)).group(0))
Exception in thread Thread-15 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
Exception in thread Thread-17 (generate_novel):
Traceback (most recent call last):
  File "C:\Prog

multithread threding count: 4
multithread threding count: 3
multithread threding count: 2
multithread threding count: 1
multithread threding count: 1
multithread threding count: 1
multithread threding count: 1
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 1
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1


Thread-71 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "C:\Users\7ross\AppData\Local\Temp\ipykernel_20980\4023973347.py", line 25, in generate_novel
    self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
NameError: name 'metadata' is not defined
Exception in thread Thread-73 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", l

multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0


    self._target(*self._args, **self._kwargs)
  File "C:\Users\7ross\AppData\Local\Temp\ipykernel_20980\4023973347.py", line 25, in generate_novel
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
Exception in thread Thread-135 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
NameError: name 'metadata' is not defined
    self._target(*self._args, **self._kwargs)
  File "C:\Users\7ross\AppData\Local\Temp\ipykernel_20980\4023973347.py", line 25, in generate_novel
Exception in thread     self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
Thread-137 (generate

...

    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
NameError: name 'metadata' is not defined
    self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self._target(*self._args, **self._kwargs)
  File "C:\Users\7ross\AppData\Local\Temp\ipykernel_20980\4023973347.py", line 25, in generate_novel
NameError: name 'metadata' is not defined
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
Exception in thread Thread-209 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", li

multithread threding count: 0
multithread threding count: 1
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 1
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 2
multithread threding count: 1
multithread threding count: 1


    self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
Exception in thread     self._target(*self._args, **self._kwargs)
  File "C:\Users\7ross\AppData\Local\Temp\ipykernel_20980\4023973347.py", line 25, in generate_novel
Thread-261 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
NameError: name 'metadata' is not defined
    self.run(

multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 2
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 1
multithread threding count: 1
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 0
multithread threding count: 1
multithread threding count: 0
multithrea

    self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
Exception in thread Thread-343 (generate_novel):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
NameError: name 'metadata' is not defined
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
    self.run()
  File "C:\Users\7ross\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
NameError: name 'metadata' is not defined
    self._target(*self._args, **self._kwargs)
  File "C:\Users\7ross\AppData\Loca